Replace attentions with dynamic adj

Возможные изменения:

Мастер-вершина

Reduce adj matrix to real vert only

e_ij - кодировка ребр; дорого по памяти

U-GNN - свертки графа с уменьшением размерности: use clustering?

RNN in GNN?

U-net cnn: 99.66%

LR: ???

pre-rnn stabilizes training

graph-nn in unet (conv+gnn)

In [1]:
import tensorflow as tf
import numpy as np
import h5py as h5
import random as rd

In [2]:
gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
h5f = '/home/ivkhar/Baikal/data/mc_baikal_norm_cut-0_ordered_equal_big.h5'
max_len = 110
batch_size = 16

# generator without shuffling
class generator_no_shuffle:
    
    def __init__(self, file, regime, batch_size, return_reminder):
        self.file = file
        self.regime = regime
        self.batch_size = batch_size
        self.return_reminder = return_reminder
        with h5.File(self.file,'r') as hf:
            self.num = hf[self.regime+'/data'].shape[0]     
        self.batch_num = self.num // self.batch_size
        if return_reminder:
            self.gen_num = self.num
        else:
            self.gen_num = self.batch_num*self.batch_size

    def __call__(self):
        start = 0
        stop = self.batch_size
        with h5.File(self.file, 'r') as hf:
            for i in range(self.batch_num):
                mask = hf[self.regime+'/mask'][start:stop]
                mask_channel = np.expand_dims( mask, axis=-1 )
                labels = hf[self.regime+'/labels_signal_noise'][start:stop]
                yield ( np.concatenate((hf[self.regime+'/data'][start:stop],mask_channel), axis=-1), 
                  labels )
                start += self.batch_size
                stop += self.batch_size
            if self.return_reminder:
                mask = hf[self.regime+'/mask'][start:stop]
                mask_channel = np.expand_dims( mask, axis=-1 )
                labels = hf[self.regime+'/labels_signal_noise'][start:stop]
                yield ( np.concatenate((hf[self.regime+'/data'][start:stop],mask_channel), axis=-1), 
                  labels )

# generator with shuffling
class generator_with_shuffle:
    
    def __init__(self, file, regime, batch_size, buffer_size, return_reminder):
        self.file = file
        self.regime = regime
        self.batch_size = batch_size
        self.return_reminder = return_reminder
        self.buffer_size = buffer_size
        with h5.File(self.file,'r') as hf:
            self.num = hf[self.regime+'/data/'].shape[0] 
        self.batch_num = (self.num-self.buffer_size) // self.batch_size
        self.last_batches_num = self.buffer_size // self.batch_size
        if return_reminder:
            self.gen_num = self.num
        else:
            self.gen_num = (self.batch_num+self.last_batches_num)*self.batch_size

    def __call__(self):
        start = self.buffer_size
        stop = self.buffer_size + self.batch_size
        with h5.File(self.file, 'r') as hf:
            mask = hf[self.regime+'/mask'][:self.buffer_size]
            mask_channel = np.expand_dims( mask, axis=-1 )
            buffer_data = np.concatenate((hf[self.regime+'/data'][:self.buffer_size],mask_channel), axis=-1)
            buffer_labels = hf[self.regime+'/labels_signal_noise'][:self.buffer_size]
            for i in range(self.batch_num):
                idxs = rd.sample( range(self.buffer_size), k=self.batch_size )
                yield ( buffer_data[idxs] , buffer_labels[idxs] )
                mask = hf[self.regime+'/mask'][start:stop]
                mask_channel = np.expand_dims( mask, axis=-1 )
                buffer_data[idxs] = np.concatenate((hf[self.regime+'/data'][start:stop],mask_channel),axis=-1)
                labels = hf[self.regime+'/labels_signal_noise'][start:stop]
                buffer_labels[idxs] = labels
                start += self.batch_size
                stop += self.batch_size
            # fill the buffer with left data, if any
            # this a bit increases buffer size and MIGT BE COSTLY
            mask = hf[self.regime+'/mask'][start:stop]
            mask_channel = np.expand_dims( mask, axis=-1 )
            buffer_data = np.concatenate( (buffer_data,np.concatenate((hf[self.regime+'/data'][start:stop],mask_channel),axis=-1)), axis=0 )
            labels = hf[self.regime+'/labels_signal_noise'][start:stop]
            buffer_labels  = np.concatenate( (buffer_labels,labels), axis=0 )
            sh_idxs = rd.sample( range(buffer_labels.shape[0]), k=buffer_labels.shape[0] )
            start = 0
            stop = self.batch_size
            for i in range(self.last_batches_num):
                idxs = sh_idxs[start:stop]
                yield ( buffer_data[idxs], buffer_labels[idxs] )
                start += self.batch_size
                stop += self.batch_size
            if self.return_reminder:
                idxs = sh_idxs[start:stop]
                yield ( buffer_data[idxs], buffer_labels[idxs])

### MADE SO BATCH SIZE IS CONSTANT AND NO REMINDER
def make_datasets(h5f,make_generator_shuffle,return_batch_reminder,train_batch_size,train_buffer_size,test_batch_size):
    # generator for training data
    if make_generator_shuffle:
        tr_generator = generator_with_shuffle(h5f,'train',train_batch_size,train_buffer_size,return_batch_reminder)
    else:
        tr_generator = generator_no_shuffle(h5f,'train',train_batch_size,return_batch_reminder)
    if return_batch_reminder:
        # size of the last batch is unknown
        tr_batch_size = None
    else:
        tr_batch_size = train_batch_size

    train_dataset = tf.data.Dataset.from_generator( tr_generator, 
                        output_signature=( tf.TensorSpec(shape=(tr_batch_size,max_len,6)), tf.TensorSpec(shape=(tr_batch_size,max_len,2))
                                         ) )

    if make_generator_shuffle:
        train_dataset = train_dataset.repeat(-1).prefetch(tf.data.AUTOTUNE)
    else:
        train_dataset = train_dataset.repeat(-1).shuffle(num_batch_shuffle)

    # generator for validation data
    te_generator = generator_no_shuffle(h5f,'test',test_batch_size,False)
    te_batch_size = tr_batch_size
    test_dataset = tf.data.Dataset.from_generator( te_generator, 
                        output_signature=( tf.TensorSpec(shape=(tr_batch_size,max_len,6)), tf.TensorSpec(shape=(tr_batch_size,max_len,2))
                                          ) )
    
    test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)

    return train_dataset, test_dataset

train_dataset, test_dataset = make_datasets(h5f,True,False,batch_size,500*batch_size,batch_size)

2022-05-05 14:39:40.099539: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-05 14:39:40.596444: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 17363 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:44:00.0, compute capability: 8.6


In [4]:
### pre-transforms nodes encodings
# returns transformed encoding plus initial
# pass empty filters to avoid new encs
# in case k!=1 neighbours effect the final encoding

# (bs,om,c) -> (bs,om,c')
class NodesEncoder(tf.keras.layers.Layer):
      
    def __init__(self, filters, kernels, activation):
        super(NodesEncoder, self).__init__()
        assert len(filters)==len(kernels)
        self.num_layers = len(filters)
        self.conv_layers = [ tf.keras.layers.Conv1D(f, k, padding='same') for (f,k) in zip(filters,kernels) ]
        self.activation = activation
        self.filters = filters
        
    def build(self, input_shape):
        if self.num_layers==0:
            self.out_encs_length = input_shape[-1]
        else:
            self.out_encs_length = self.filters[-1]+input_shape[-1]

    def call(self, x, training=False):
        init_x = x
        for conv in self.conv_layers:
            x = conv(x)
            x = self.activation(x)
            x = tf.concat((x,init_x),axis=-1)
        return x

In [5]:
# creates messages for pairs of nodes
# k=(1,1) reduces to usual dense

# (bs,om,om,c) -> (bs,om,om,c')
class MessageCreator(tf.keras.layers.Layer):
    
    def __init__(self, filters, kernels, activation):
        super(MessageCreator, self).__init__()
        self.conv_layers = [ tf.keras.layers.Conv2D(f, k, padding='same') for (f,k) in zip(filters,kernels) ]
        self.activation = activation

    def call(self, x):
        for lr in self.conv_layers:
            x = lr(x)
            x = self.activation(x)
        return x

In [6]:
### message parser
# reduces all messages to single message

# (bs,om,om,c) -> (bs,om,c')
class MessageParser(tf.keras.layers.Layer):
  
    # filters, kernels, strides - sequances
    def __init__(self, filters, kernels, strides, units, activation, take_mean):
        super(MessageParser, self).__init__()
        assert len(filters)==len(kernels)
        assert len(filters)==len(strides)
        self.conv_layers = [ tf.keras.layers.Conv2D(f, k, strides=s, padding='same') for (f,k,s) in zip(filters,kernels,strides) ]
        self.dence_layers = [ tf.keras.layers.Dense(un) for un in units ]
        self.activation = activation
        self.max_poolings = [  tf.keras.layers.MaxPooling2D(pool_size=(1,2)) for f in filters ]
        self.take_mean = take_mean

    def call(self, x, training=False):
        for (conv,mp) in zip(self.conv_layers,self.max_poolings):
            x = conv(x)
            x = self.activation(x)
            x = mp(x)
        # (bs,om,om',c')
        if self.take_mean:
            x = tf.math.reduce_mean(x, axis=2) # (bs,om,c')
        else:
            x = tf.reshape(x, (x.shape[0],x.shape[1],-1)) # (bs,om,c')
        for de in self.dence_layers:
            x = de(x)
            x = self.activation(x)
        return x

In [7]:
### state updater

# (bs,om,c) -> (bs,om,c')
class StateUpdater(tf.keras.layers.Layer):
  
    def __init__(self, units, activations):
        super(StateUpdater, self).__init__()
        self.dense_layers = [ tf.keras.layers.Dense(u) for u in units ]
        self.activations = [ act for act in activations ]

    def call(self, x, training=False):
        for (dense,ac) in zip(self.dense_layers,self.activations):
            x = dense(x)
            x = ac(x)
        return x

In [8]:
### adj updater

# (bs,om,om,c)x(bs,om,om,1) -> (bs,om,om,1)
class AdjUpdater(tf.keras.layers.Layer):
  
    def __init__(self, v_filters, v_kernels, v_activation, j_filters, j_kernels, j_activations):
        super(AdjUpdater, self).__init__()
        assert len(v_filters)==len(v_kernels)
        assert len(j_filters)==len(j_kernels)
        self.conv_layers_vert = [ tf.keras.layers.Conv2D(f, k, padding='same') for (f,k) in zip(v_filters,v_kernels) ]
        self.conv_layers_joint = [ tf.keras.layers.Conv2D(f, k, padding='same') for (f,k) in zip(j_filters,j_kernels) ]
        self.v_activation = v_activation
        self.j_activations = j_activations

    def call(self, vert, adj):
        # preprocess verticies
        x = vert
        for lr in self.conv_layers_vert:
            x = lr(x)
            x = self.v_activation(x)
        # update
        x = tf.concat((x,adj), axis=-1)
        for (lr,ac) in zip(self.conv_layers_joint,self.j_activations):
            x = lr(x)
            x = ac(x)
        return x

In [9]:
# MAKE TF WHERE WITH ADJ ???
class GraphStepLayer(tf.keras.Model):
    
    def __init__(self, nodes_encoder, message_creator, message_parser, state_updater, adj_updater):
        super(GraphStepLayer, self).__init__()
        self.nodes_encoder = nodes_encoder
        self.message_creator = message_creator
        self.message_parser = message_parser
        self.state_updater = state_updater
        self.adj_updater = adj_updater
        self.pos_channel = np.expand_dims(np.expand_dims(np.arange(-1,1,2/max_len),axis=-1),axis=0)

    def build(self, input_shape):
        self.nodes_encoder.build(input_shape)
        
    # (bs,om,c) -> (bs,om,om,2c)
    def make_pairs(self, vert):
        vert_exp_1 = tf.expand_dims(vert,axis=1) # (bs,1,om,c)
        vert_exp_2 = tf.expand_dims(vert,axis=2) # (bs,om,1,c)
        helper = tf.fill( (vert.shape[0],vert.shape[1],vert.shape[1],1), 1. )
        rel_vert = helper*vert_exp_2 # (bs, om, om_targ, c)
        base_vert = helper*vert_exp_1 # (bs, om_base, om, c)
        pairs = tf.concat((rel_vert,base_vert),axis=-1) # (bs, om_base, om_target, 2c)
        return pairs
        
    def call(self, inputes):
        (vert, adj) = inputes
        # encode nodes for message passing
        node_encs = self.nodes_encoder(vert) # (bs,om,c)
        # prepare messages
        # add position channel
        pos_encs = np.repeat( self.pos_channel, vert.shape[0], axis=0 )
        node_encs_pos = tf.concat( (node_encs,pos_encs), axis=-1 ) # (bs,om,c)
        pairs = self.make_pairs(node_encs_pos) # (b,om,om,2c)
        # use adj as attention
        #messages = tf.where(init_adj, self.message_creator(adj*pairs), [0]) # (b,om,om,c')
        messages = self.message_creator(adj*pairs) # (b,om,om,c')
        # prepare single message
        mess = self.message_parser(messages) # (b,om,c)
        # update states
        update_from = tf.concat((node_encs,mess), axis=-1) # (b, om, ch')
        out_encs = self.state_updater( update_from ) # (b, om, ch)
        # update adj matrix
        pairs_n = self.make_pairs(out_encs)
        adj_n = self.adj_updater(pairs_n,adj)
        return (out_encs, adj_n)

In [10]:
# graph layer with residual connection
class GraphResLayer(tf.keras.Model):
    
    def __init__(self, nodes_encoders, message_creators, message_parsers, state_updaters, adj_updaters):
        super(GraphResLayer, self).__init__()
        self.gr_layers = []
        for i in range(2):
            gr_layer = GraphStepLayer( nodes_encoders[i], message_creators[i], message_parsers[i], state_updaters[i], adj_updaters[i])
            self.gr_layers.append(gr_layer)
        
    def call(self, inputs):
        (vert, adj) = inputs
        # pass through fist layer
        (vert1, adj1) = self.gr_layers[0]((vert, adj))
        # second
        (vert2, adj2) = self.gr_layers[1]((vert1, adj1))
        # concat
        vert_res = tf.concat((vert1,vert2), axis=-1)
        return (vert_res, (adj1+adj2)/2 )

In [11]:
# make graph model
class GraphLikeModel(tf.keras.Model):
    
    def __init__(self, pre_layer, gnn_layers, post_layer, k_nearest):
        super(GraphLikeModel, self).__init__()
        self.pre_layer = pre_layer
        self.graph_layers = gnn_layers
        self.post_layer = post_layer
        if k_nearest>0:
            te = [ np.expand_dims(np.eye(max_len),axis=0)]
            for i in range(1,k_nearest):
                te.append(np.expand_dims(np.eye(max_len, k=i, dtype=np.float32),axis=0))
                te.append(np.expand_dims(np.eye(max_len, k=-i, dtype=np.float32),axis=0))
            self.full_adj = tf.expand_dims( np.sum( np.concatenate(te, axis=0), axis=0 ), axis=-1)
        else:
            self.full_adj = tf.fill((1,max_len,max_len,1), 1.)
        self.full_adj = tf.cast(self.full_adj, tf.float32)
              
    def call(self, x):
        # readout mask and make adj matrix
        mask = tf.cast(x[:,:,-1:], tf.float32) # (bs,om,1)
        adj = self.full_adj*tf.expand_dims(mask,axis=1)*tf.expand_dims(mask,axis=2) # (bs,om,om,1)
        # normilize, not proper at first layer
        adj = tf.keras.activations.softmax(adj)
        # preprocess inputs
        x = self.pre_layer(x)
        # pass via graph layers
        for gr_lr in self.graph_layers:
            (x, adj) = gr_lr((x,adj))
        # postprocess
        x = self.post_layer(x)
        # make preds
        preds = tf.where( tf.cast(mask,bool), x, tf.constant([0.,1.]) )
        return preds

In [12]:
def compile_model(model, lr):
    loss = tf.keras.losses.CategoricalCrossentropy()
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=True)
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    return model

In [13]:
selu = tf.keras.activations.selu
relu = tf.keras.activations.relu
sigmoid = tf.keras.activations.sigmoid
softmax = tf.keras.activations.softmax

depth = 3
filts = 128

# create nodes encoders
node_enc_filters = [[filts] for i in range(depth)]
node_enc_kernels =  [[1] for i in range(depth)]
node_enc_activation = selu
nodes_encoders = []
for (filters,kernels) in zip(node_enc_filters,node_enc_kernels):
    nodes_encoders.append( [NodesEncoder(filters,kernels,node_enc_activation),NodesEncoder(filters,kernels,node_enc_activation)] )

# make message creators
# 2d kernels
mess_cr_filters = [[filts] for i in range(depth)]
mess_cr_kernels = [[(1,1)] for i in range(depth)]
mess_cr_activation = selu
mess_crs = []
for (filters,kernels) in zip(mess_cr_filters,mess_cr_kernels):
    mess_crs.append( [MessageCreator(filters,kernels,mess_cr_activation),MessageCreator(filters,kernels,mess_cr_activation)] )

# create message parsers
# 2d kernels and strides
mess_pars_filters = [32]
mess_pars_kernels = [(1,5)]
mess_pars_strides = [(1,2)]
mess_pars_units = [[filts] for i in range(depth)]
mess_pass_activation = selu
mess_pars_take_mean = False
mess_parsers = []
for units in mess_pars_units:
    mess_parsers.append( [MessageParser(mess_pars_filters,mess_pars_kernels,mess_pars_strides,units,mess_pass_activation,mess_pars_take_mean),
                         MessageParser(mess_pars_filters,mess_pars_kernels,mess_pars_strides,units,mess_pass_activation,mess_pars_take_mean)] )

# make adj updateras
# 2d kernels
adj_v_filters = [[32] for i in range(depth)]
adj_v_kernels = [[(1,1)] for i in range(depth)]
adj_v_activation = selu
adj_j_filters = [[16,1] for i in range(depth)]
adj_j_kernels = [[(1,1),(1,1)] for i in range(depth)]
att_j_activations = [[selu,sigmoid] for i in range(depth)]
adj_upds = []
for (v_filters,v_kernels,j_filters,j_kernels,j_act) in zip(adj_v_filters,adj_v_kernels,adj_j_filters,adj_j_kernels,att_j_activations):
    adj_upds.append( [AdjUpdater(v_filters,v_kernels,adj_v_activation,j_filters,j_kernels,j_act),AdjUpdater(v_filters,v_kernels,adj_v_activation,j_filters,j_kernels,j_act)] )

# make state updaters
state_upd_units = [[(3*filts//2)] for i in range(depth)]
state_upd_activations = [[selu] for i in range(depth)]
state_ups = []
for (units,act) in zip(state_upd_units,state_upd_activations):
    state_ups.append( [StateUpdater(units,act),StateUpdater(units,act)] )

# make graph layers
gr_layers = []
for (node_enc,mess_cr,mess_pars,adj_upd,state_upd) in zip(nodes_encoders,mess_crs,mess_parsers,adj_upds,state_ups):
    gr_layers.append( GraphResLayer(node_enc, mess_cr, mess_pars, state_upd, adj_upd) )
    
# make pre-layers
pre_layer = tf.identity
#pre_layer = tf.keras.layers.LSTM(32, activation='tanh', recurrent_activation='sigmoid', return_sequences=True)

# make post_layers
post_layer = tf.keras.layers.Dense(2, activation=softmax)

# make gnn
gnn = GraphLikeModel( pre_layer, gr_layers, post_layer, -1)
gnn = compile_model(gnn, 0.0006)

print(len(gr_layers))
# grads do not exist for the last adj updater as it does not effect prediction

3


In [14]:
# carefull with lr and bs

In [15]:
gnn.fit(train_dataset, steps_per_epoch=2500, validation_steps=500, epochs=25, validation_data=test_dataset, verbose=1)

Epoch 1/25


2022-05-05 14:40:06.249667: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8301
2022-05-05 14:40:07.452611: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-05-05 14:40:07.590259: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


2500/2500 [==============================] - 526s 207ms/step - loss: 0.0437 - accuracy: 0.9824 - val_loss: 0.0277 - val_accuracy: 0.9887
Epoch 2/25
2500/2500 [==============================] - 517s 207ms/step - loss: 0.0211 - accuracy: 0.9915 - val_loss: 0.0223 - val_accuracy: 0.9910
Epoch 3/25
2500/2500 [==============================] - 517s 207ms/step - loss: 0.0169 - accuracy: 0.9933 - val_loss: 0.0211 - val_accuracy: 0.9914
Epoch 4/25
2500/2500 [==============================] - 517s 207ms/step - loss: 0.0151 - accuracy: 0.9940 - val_loss: 0.0167 - val_accuracy: 0.9934
Epoch 5/25
2500/2500 [==============================] - 517s 207ms/step - loss: 0.0161 - accuracy: 0.9938 - val_loss: 0.0164 - val_accuracy: 0.9935
Epoch 6/25
2500/2500 [==============================] - 517s 207ms/step - loss: 0.0128 - accuracy: 0.9950 - val_loss: 0.0156 - val_accuracy: 0.9939
Epoch 7/25
2500/2500 [==============================] - 517s 207ms/step - loss: 0.0123 - accuracy: 0.9952 - val_loss: 0.014

KeyboardInterrupt: 

In [185]:
# with rnn, k=2
gnn.fit(train_dataset, steps_per_epoch=2500, validation_steps=500, epochs=10, validation_data=test_dataset, verbose=1)

Epoch 1/10
2500/2500 [==============================] - 235s 92ms/step - loss: 0.0461 - accuracy: 0.9805 - val_loss: 0.0293 - val_accuracy: 0.9880
Epoch 2/10
2500/2500 [==============================] - 228s 91ms/step - loss: 0.0241 - accuracy: 0.9902 - val_loss: 0.0235 - val_accuracy: 0.9905
Epoch 3/10
2500/2500 [==============================] - 231s 92ms/step - loss: 0.0206 - accuracy: 0.9917 - val_loss: 0.0211 - val_accuracy: 0.9916
Epoch 4/10
2500/2500 [==============================] - 231s 92ms/step - loss: 0.0191 - accuracy: 0.9923 - val_loss: 0.0215 - val_accuracy: 0.9913
Epoch 5/10
2500/2500 [==============================] - 232s 93ms/step - loss: 0.0183 - accuracy: 0.9926 - val_loss: 0.0205 - val_accuracy: 0.9918
Epoch 6/10
2500/2500 [==============================] - 230s 92ms/step - loss: 0.0173 - accuracy: 0.9930 - val_loss: 0.0208 - val_accuracy: 0.9916
Epoch 7/10
2500/2500 [==============================] - 230s 92ms/step - loss: 0.0168 - accuracy: 0.9932 - val_loss: 0

In [180]:
# without rnn, k=2, ident@enc
gnn.fit(train_dataset, steps_per_epoch=2500, validation_steps=500, epochs=10, validation_data=test_dataset, verbose=1)

Epoch 1/10
2500/2500 [==============================] - 215s 84ms/step - loss: 0.0376 - accuracy: 0.9842 - val_loss: 0.0271 - val_accuracy: 0.9887
Epoch 2/10
2500/2500 [==============================] - 210s 84ms/step - loss: 0.0223 - accuracy: 0.9909 - val_loss: 0.0225 - val_accuracy: 0.9909
Epoch 3/10
2500/2500 [==============================] - 211s 84ms/step - loss: 0.0185 - accuracy: 0.9927 - val_loss: 0.0200 - val_accuracy: 0.9921
Epoch 4/10
2500/2500 [==============================] - 210s 84ms/step - loss: 0.0163 - accuracy: 0.9936 - val_loss: 0.0190 - val_accuracy: 0.9926
Epoch 5/10
2500/2500 [==============================] - 210s 84ms/step - loss: 0.0153 - accuracy: 0.9939 - val_loss: 0.0178 - val_accuracy: 0.9929
Epoch 6/10
2500/2500 [==============================] - 210s 84ms/step - loss: 0.0142 - accuracy: 0.9943 - val_loss: 0.0172 - val_accuracy: 0.9932
Epoch 7/10
2500/2500 [==============================] - 210s 84ms/step - loss: 0.0137 - accuracy: 0.9946 - val_loss: 0

In [195]:
# without rnn, k=2
gnn.fit(train_dataset, steps_per_epoch=2500, validation_steps=500, epochs=10, validation_data=test_dataset, verbose=1)

Epoch 1/10
2500/2500 [==============================] - 214s 84ms/step - loss: 0.0389 - accuracy: 0.9839 - val_loss: 0.0268 - val_accuracy: 0.9893
Epoch 2/10
2500/2500 [==============================] - 215s 86ms/step - loss: 0.0200 - accuracy: 0.9922 - val_loss: 0.0197 - val_accuracy: 0.9924
Epoch 3/10
2500/2500 [==============================] - 208s 83ms/step - loss: 0.0159 - accuracy: 0.9938 - val_loss: 0.0187 - val_accuracy: 0.9928
Epoch 4/10
2500/2500 [==============================] - 208s 83ms/step - loss: 0.0142 - accuracy: 0.9945 - val_loss: 0.0164 - val_accuracy: 0.9935
Epoch 5/10
2500/2500 [==============================] - 209s 84ms/step - loss: 0.0133 - accuracy: 0.9948 - val_loss: 0.0157 - val_accuracy: 0.9938
Epoch 6/10
2500/2500 [==============================] - 209s 84ms/step - loss: 0.0122 - accuracy: 0.9952 - val_loss: 0.0143 - val_accuracy: 0.9945
Epoch 7/10
2500/2500 [==============================] - 240s 96ms/step - loss: 0.0117 - accuracy: 0.9954 - val_loss: 0

In [187]:
# without rnn, k=-1, ident@enc
gnn.fit(train_dataset, steps_per_epoch=2500, validation_steps=500, epochs=10, validation_data=test_dataset, verbose=1)

Epoch 1/10
2500/2500 [==============================] - 214s 84ms/step - loss: 0.0389 - accuracy: 0.9837 - val_loss: 0.0291 - val_accuracy: 0.9879
Epoch 2/10
2500/2500 [==============================] - 211s 84ms/step - loss: 0.0221 - accuracy: 0.9909 - val_loss: 0.0226 - val_accuracy: 0.9912
Epoch 3/10
2500/2500 [==============================] - 211s 84ms/step - loss: 0.0182 - accuracy: 0.9928 - val_loss: 0.0217 - val_accuracy: 0.9915
Epoch 4/10
2500/2500 [==============================] - 211s 84ms/step - loss: 0.0165 - accuracy: 0.9935 - val_loss: 0.0186 - val_accuracy: 0.9927
Epoch 5/10
2500/2500 [==============================] - 211s 85ms/step - loss: 0.0154 - accuracy: 0.9939 - val_loss: 0.0176 - val_accuracy: 0.9931
Epoch 6/10
2500/2500 [==============================] - 211s 85ms/step - loss: 0.0144 - accuracy: 0.9942 - val_loss: 0.0168 - val_accuracy: 0.9933
Epoch 7/10
2500/2500 [==============================] - 211s 84ms/step - loss: 0.0141 - accuracy: 0.9944 - val_loss: 0

In [189]:
# without rnn, k=4
gnn.fit(train_dataset, steps_per_epoch=2500, validation_steps=500, epochs=10, validation_data=test_dataset, verbose=1)

Epoch 1/10
2500/2500 [==============================] - 221s 87ms/step - loss: 0.0461 - accuracy: 0.9808 - val_loss: 0.0350 - val_accuracy: 0.9851
Epoch 2/10
2500/2500 [==============================] - 215s 86ms/step - loss: 0.0272 - accuracy: 0.9886 - val_loss: 0.0273 - val_accuracy: 0.9887
Epoch 3/10
2458/2500 [============================>.] - ETA: 3s - loss: 0.0228 - accuracy: 0.9906

KeyboardInterrupt: 

In [191]:
# without rnn, k=-1, softmax 
gnn.fit(train_dataset, steps_per_epoch=2500, validation_steps=500, epochs=10, validation_data=test_dataset, verbose=1)

Epoch 1/10
2500/2500 [==============================] - 219s 86ms/step - loss: 0.0443 - accuracy: 0.9812 - val_loss: 0.0390 - val_accuracy: 0.9835
Epoch 2/10
 986/2500 [==========>...................] - ETA: 2:01 - loss: 0.0290 - accuracy: 0.9876

KeyboardInterrupt: 

In [193]:
# without rnn, k=-1, sigmoid 
gnn.fit(train_dataset, steps_per_epoch=2500, validation_steps=500, epochs=10, validation_data=test_dataset, verbose=1)

Epoch 1/10
2500/2500 [==============================] - 212s 84ms/step - loss: 0.0393 - accuracy: 0.9835 - val_loss: 0.0282 - val_accuracy: 0.9890
Epoch 2/10
2500/2500 [==============================] - 208s 83ms/step - loss: 0.0218 - accuracy: 0.9911 - val_loss: 0.0233 - val_accuracy: 0.9906
Epoch 3/10
2500/2500 [==============================] - 208s 83ms/step - loss: 0.0188 - accuracy: 0.9924 - val_loss: 0.0203 - val_accuracy: 0.9918
Epoch 4/10
2500/2500 [==============================] - 207s 83ms/step - loss: 0.0168 - accuracy: 0.9932 - val_loss: 0.0201 - val_accuracy: 0.9921
Epoch 5/10
2500/2500 [==============================] - 208s 83ms/step - loss: 0.0156 - accuracy: 0.9938 - val_loss: 0.0176 - val_accuracy: 0.9931
Epoch 6/10
2500/2500 [==============================] - 208s 83ms/step - loss: 0.0143 - accuracy: 0.9943 - val_loss: 0.0175 - val_accuracy: 0.9931
Epoch 7/10
2500/2500 [==============================] - 208s 83ms/step - loss: 0.0137 - accuracy: 0.9946 - val_loss: 0

In [ ]:
# compare with 99.23@8